In [608]:
import requests

url = "https://fbref.com/en/comps/9/2019-2020/stats/2019-2020-Premier-League-Stats"
response = requests.get(url)
response

<Response [200]>

In [609]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import html

In [610]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
  <link href="https://cdn.ssref.net/req/202311031" rel="dns-prefetch"/>
  Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0)
  <script async="true" type="text/javascript">
   (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME =

In [611]:
bs1 = BeautifulSoup(response.content, 'html.parser')
text_of_bs1 = str(bs1)
usable_bs2 = text_of_bs1.replace("<!--"," ").replace("-->", " ")

In [612]:
soup = BeautifulSoup(usable_bs2, 'html.parser')

In [613]:
url_list = soup.find_all('a')
url_list = [url for url in url_list if str(url).find('player') > 0]
url_list = [url for url in url_list if str(url).find('matchlogs') < 0]
url_list = [str(url) for url in url_list]
url_list

['<a href="/en/players/">Players</a>',
 '<a href="/en/players/45963054/Jamie-Vardy">Jamie Vardy</a>',
 '<a href="/en/players/e46012d4/Kevin-De-Bruyne">Kevin De Bruyne</a>',
 '<a href="/en/players/3bb7b8b4/Ederson">Ederson</a>',
 '<a href="/en/players/5f09991f/Patrick-van-Aanholt">Patrick van Aanholt</a>',
 '<a href="/en/players/774cf58b/Max-Aarons">Max Aarons</a>',
 '<a href="/en/players/f586779e/Tammy-Abraham">Tammy Abraham</a>',
 '<a href="/en/players/f2bf1b0f/Che-Adams">Che Adams</a>',
 '<a href="/en/players/f76e6b4e/Adrian">Adrián</a>',
 '<a href="/en/players/4d034881/Sergio-Aguero">Sergio Agüero</a>',
 '<a href="/en/players/2b1c4abd/Albian-Ajeti">Albian Ajeti</a>',
 '<a href="/en/players/eaeca114/Nathan-Ake">Nathan Aké</a>',
 '<a href="/en/players/b827d5b3/Marc-Albrighton">Marc Albrighton</a>',
 '<a href="/en/players/f7d50789/Toby-Alderweireld">Toby Alderweireld</a>',
 '<a href="/en/players/cd1acf9d/Trent-Alexander-Arnold">Trent Alexander-Arnold</a>',
 '<a href="/en/players/7a2e46

In [614]:
url_id_list = []
tag_list = []
excluded_indexes = []
for i in range(0, len(url_list)):
    if len(url_list[i]) > 40 and len(url_list[i]) < 100:
        url_id_list.append(url_list[i].split('"')[1].split('/')[3])
        tag_list.append(url_list[i].split('"')[1].split('/')[4])
    else:
        excluded_indexes.append(i)

len(tag_list)

540

In [615]:
excluded_indexes

[0, 526, 542, 543]

In [616]:
tag_list = tag_list[0:(excluded_indexes[1]-1)]
url_id_list = url_id_list[0:(excluded_indexes[1]-1)]

In [617]:
url_id_values = pd.Series(url_id_list)
tag_values = pd.Series(tag_list)
name_values = pd.Series(name_list)
df = pd.DataFrame()
df['url_id'] = url_id_values
df['tag'] = tag_values
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df

,url_id,tag
0,45963054,Jamie-Vardy
1,e46012d4,Kevin-De-Bruyne
2,3bb7b8b4,Ederson
3,5f09991f,Patrick-van-Aanholt
4,774cf58b,Max-Aarons
...,...,...
510,b2bc3b1f,Wilfried-Zaha
511,2d5f03bf,Christoph-Zimmermann
512,51cf8561,Oleksandr-Zinchenko
513,d2d74d9c,Richairo-Zivkovic


In [618]:
player_id_df = pd.read_csv('../data/2019-20/player_idlist.csv')

In [619]:
player_id_df['full_name'] = player_id_df['first_name'] + ' ' + player_id_df['second_name']

In [620]:
# for name in df['tag']:
#     sp = name.split('-')
#     if all(x in player_id_df['full_name'] for x in sp) == True:
        

In [621]:
df['key'] = 0
player_id_df['key'] = 0

In [622]:
df

,url_id,tag,key
0,45963054,Jamie-Vardy,0
1,e46012d4,Kevin-De-Bruyne,0
2,3bb7b8b4,Ederson,0
3,5f09991f,Patrick-van-Aanholt,0
4,774cf58b,Max-Aarons,0
...,...,...,...
510,b2bc3b1f,Wilfried-Zaha,0
511,2d5f03bf,Christoph-Zimmermann,0
512,51cf8561,Oleksandr-Zinchenko,0
513,d2d74d9c,Richairo-Zivkovic,0


In [623]:
merge_df = pd.merge(df, player_id_df, on='key')
def filter_name(row):
    sp = row.tag.split('-')
    if all([x in row.full_name for x in sp]):
        return True
    else:
        return False

In [597]:
merge_df['keep'] = merge_df.apply(filter_name, axis = 1)

In [598]:
merge_df = merge_df[merge_df['keep']]

In [599]:
merge_df = merge_df.reset_index(drop = True)

In [600]:
duplicated_df = merge_df[merge_df['tag'].duplicated(keep = False)]

In [601]:
duplicated_df = duplicated_df.reset_index()
duplicated_df

,index,url_id,tag,key,first_name,second_name,id,full_name,keep
0,42,ab4b3e35,Bernard,0,Bernardo,Fernandes da Silva Junior,39,Bernardo Fernandes da Silva Junior,True
1,43,ab4b3e35,Bernard,0,Bernard,Anício Caldeira Duarte,152,Bernard Anício Caldeira Duarte,True
2,44,ab4b3e35,Bernard,0,Bernardo Mota,Veiga de Carvalho e Silva,218,Bernardo Mota Veiga de Carvalho e Silva,True
3,45,ab4b3e35,Bernard,0,Bernardo,Costa Da Rosa,595,Bernardo Costa Da Rosa,True
4,46,ab4b3e35,Bernard,0,Bernard,Ashley-Seal,559,Bernard Ashley-Seal,True
5,47,17370b95,Bernardo,0,Bernardo,Fernandes da Silva Junior,39,Bernardo Fernandes da Silva Junior,True
6,48,17370b95,Bernardo,0,Bernardo Mota,Veiga de Carvalho e Silva,218,Bernardo Mota Veiga de Carvalho e Silva,True
7,49,17370b95,Bernardo,0,Bernardo,Costa Da Rosa,595,Bernardo Costa Da Rosa,True
8,134,b853e0ad,Fred,0,Frederico,Rodrigues de Paula Santos,244,Frederico Rodrigues de Paula Santos,True
9,135,b853e0ad,Fred,0,Frederick,Woodman,264,Frederick Woodman,True


In [602]:
def filter_name_dup(row):
    split1 = row.fbref_name.split()
    split2 = row.full_name.split()
    if all([x in row.full_name for x in split1]) or all([x in row.fbref_name for x in split2]):
        return True
    else:
        return False

In [603]:
name_list = []
for x in range(0,len(duplicated_df)):
    url_dup = f"https://fbref.com/en/players/{duplicated_df.iloc[x]['url_id']}/matchlogs/2021-2022/c9/summary/{duplicated_df.iloc[x]['tag']}-Match-Log"
    reply = requests.get(url_dup)
    soup2 = BeautifulSoup(reply.content, 'html.parser')
    name = soup2.select('div:nth-child(2) > p:nth-child(2) > strong')[0].text
    name_list.append(name)

duplicated_df['fbref_name'] = name_list
duplicated_df['keep'] = duplicated_df.apply(filter_name_dup, axis = 1)
duplicated_df[duplicated_df['keep'] == False]

,index,url_id,tag,key,first_name,second_name,id,full_name,keep,fbref_name
0,42,ab4b3e35,Bernard,0,Bernardo,Fernandes da Silva Junior,39,Bernardo Fernandes da Silva Junior,False,Bernard Anício Caldeira Duarte
2,44,ab4b3e35,Bernard,0,Bernardo Mota,Veiga de Carvalho e Silva,218,Bernardo Mota Veiga de Carvalho e Silva,False,Bernard Anício Caldeira Duarte
3,45,ab4b3e35,Bernard,0,Bernardo,Costa Da Rosa,595,Bernardo Costa Da Rosa,False,Bernard Anício Caldeira Duarte
4,46,ab4b3e35,Bernard,0,Bernard,Ashley-Seal,559,Bernard Ashley-Seal,False,Bernard Anício Caldeira Duarte
6,48,17370b95,Bernardo,0,Bernardo Mota,Veiga de Carvalho e Silva,218,Bernardo Mota Veiga de Carvalho e Silva,False,Bernardo Fernandes da Silva Junior
7,49,17370b95,Bernardo,0,Bernardo,Costa Da Rosa,595,Bernardo Costa Da Rosa,False,Bernardo Fernandes da Silva Junior
9,135,b853e0ad,Fred,0,Frederick,Woodman,264,Frederick Woodman,False,Frederico Rodrigues Santos
10,136,b853e0ad,Fred,0,Ryan,Fredericks,380,Ryan Fredericks,False,Frederico Rodrigues Santos
11,310,3ca7254a,Pedro,0,Pedro,Rodríguez Ledesma,114,Pedro Rodríguez Ledesma,False,Pedro Eliezer Rodríguez
12,311,3ca7254a,Pedro,0,João Pedro Cavaco,Cancelo,518,João Pedro Cavaco Cancelo,False,Pedro Eliezer Rodríguez


In [604]:
for x in duplicated_df[duplicated_df['keep'] == False]['index']:
    merge_df = merge_df.drop(index=[x])

In [605]:
merge_df[merge_df['tag'].duplicated(keep = False)]

,url_id,tag,key,first_name,second_name,id,full_name,keep


In [577]:
merge_df = merge_df.drop(index=[103])

In [606]:
merge_df = merge_df.reset_index(drop = True)

In [607]:
merge_df

,url_id,tag,key,first_name,second_name,id,full_name,keep
0,45963054,Jamie-Vardy,0,Jamie,Vardy,166,Jamie Vardy,True
1,e46012d4,Kevin-De-Bruyne,0,Kevin,De Bruyne,215,Kevin De Bruyne,True
2,3bb7b8b4,Ederson,0,Ederson,Santana de Moraes,212,Ederson Santana de Moraes,True
3,5f09991f,Patrick-van-Aanholt,0,Patrick,van Aanholt,123,Patrick van Aanholt,True
4,774cf58b,Max-Aarons,0,Maximillian,Aarons,274,Maximillian Aarons,True
...,...,...,...,...,...,...,...,...
433,b2bc3b1f,Wilfried-Zaha,0,Wilfried,Zaha,133,Wilfried Zaha,True
434,2d5f03bf,Christoph-Zimmermann,0,Christoph,Zimmermann,275,Christoph Zimmermann,True
435,51cf8561,Oleksandr-Zinchenko,0,Oleksandr,Zinchenko,206,Oleksandr Zinchenko,True
436,d2d74d9c,Richairo-Zivkovic,0,Richairo,Zivkovic,622,Richairo Zivkovic,True


In [579]:
merge_df.to_csv('20_21_merged.csv')

In [ ]:
merge_df[merge_df['tag'] == 'Rodri']

,url_id,tag,key,first_name,second_name,id,full_name,keep


In [ ]:
summary_list = []
passing_list = []
gca_list = []
for x in range(0, (len(df)-1)):
    summary_list.append(f"https://fbref.com/en/players/{df.iloc[x]['url_id']}/matchlogs/2020-2021/c9/summary/{df.iloc[x]['tag']}-Match-Log")
    passing_list.append(f"https://fbref.com/en/players/{df.iloc[x]['url_id']}/matchlogs/2020-2021/c9/passing/{df.iloc[x]['tag']}-Match-Log")
    gca_list.append(f"https://fbref.com/en/players/{df.iloc[x]['url_id']}/matchlogs/2020-2021/c9/gca/{df.iloc[x]['tag']}-Match-Log")


In [625]:
ft = pd.read_html('https://fbref.com/en/players/cd1acf9d/matchlogs/2020-2021/c9/summary/Trent-Alexander-Arnold-Match-Log')[0]
ft

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                 Date                Day              Round   
0          2020-09-12                Sat        Matchweek 1   
1          2020-09-20                Sun        Matchweek 2   
2          2020-09-28                Mon        Matchweek 3   
3          2020-10-04                Sun        Matchweek 4   
4          2020-10-17                Sat        Matchweek 5   
5          2020-10-24                Sat        Matchweek 6   
6          2020-10-31                Sat        Matchweek 7   
7          2020-11-08                Sun        Matchweek 8   
8                 NaN                NaN                NaN   
9          2020-12-06                Sun       Matchweek 11   
10         2020-12-13                Sun       Matchweek 12   
11         2020-12-16                Wed       Matchweek 13   
12         2020-12-19                Sat       Matchweek 14   
13         2020-12-27                Sun       Matchweek 15   
14         2020-12-30                Wed       Matchweek 16   
15         2021-01-04                Mon       Matchweek 17   
16         2021-01-17                Sun       Matchweek 19   
17         2021-01-21                Thu       Matchweek 18   
18         2021-01-28                Thu       Matchweek 20   
19         2021-01-31                Sun       Matchweek 21   
20         2021-02-03                Wed       Matchweek 22   
21         2021-02-07                Sun       Matchweek 23   
22         2021-02-13                Sat       Matchweek 24   
23         2021-02-20                Sat       Matchweek 25   
24         2021-02-28                Sun       Matchweek 26   
25         2021-03-04                Thu       Matchweek 29   
26         2021-03-07                Sun       Matchweek 27   
27         2021-03-15                Mon       Matchweek 28   
28         2021-04-03                Sat       Matchweek 30   
29         2021-04-10                Sat       Matchweek 31   
30         2021-04-19                Mon       Matchweek 32   
31         2021-04-24                Sat       Matchweek 33   
32         2021-05-08                Sat       Matchweek 35   
33         2021-05-13                Thu       Matchweek 34   
34         2021-05-16                Sun       Matchweek 36   
35         2021-05-19                Wed       Matchweek 37   
36         2021-05-23                Sun       Matchweek 38   
37                NaN                NaN                NaN   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                Venue             Result              Squad   
0                Home              W 4–3          Liverpool   
1                Away              W 2–0          Liverpool   
2                Home              W 3–1          Liverpool   
3                Away              L 2–7          Liverpool   
4                Away              D 2–2          Liverpool   
5                Home              W 2–1          Liverpool   
6                Home              W 2–1          Liverpool   
7                Away              D 1–1          Liverpool   
8                 NaN                NaN                NaN   
9                Home              W 4–0          Liverpool   
10               Away              D 1–1          Liverpool   
11               Home              W 2–1          Liverpool   
12               Away              W 7–0          Liverpool   
13               Home              D 1–1          Liverpool   
14               Away              D 0–0          Liverpool   
15               Away              L 0–1          Liverpool   
16               Home              D 0–0          Liverpool   
17               Home              L 0–1          Liverpool   
18               Away              W 3–1          Liverpool   
19               Away              W 3–1          Liverpool   
20               Home              L 0–1          Liverpool   
21               Home            

In [ ]:
ft.columns = [col[1] for col in ft.columns]

In [ ]:
ft = ft.dropna()

In [ ]:
ft.columns


Index(['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start',
       'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR',
       'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xAG', 'SCA', 'GCA',
       'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries', 'PrgC', 'Att', 'Succ',
       'Match Report'],
      dtype='object')